In [76]:
import tensorflow as tf
import tensorflow_hub as hub
import os, pandas as pd
import re

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

file_list=os.listdir('../../dataset/df_train.pkl.gz/')
df_train=pd.DataFrame()
for file in file_list:
    df_train=pd.concat([df_train, 
                        pd.read_pickle('../../dataset/df_train.pkl.gz/'+file, compression='gzip')])
df_train['mission_prgrm']=df_train['mission']+'; '+df_train['prgrm_dsc']

small_num=0
while small_num<100: # Make sure each category has at least 100 records.
    trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(40000)
    small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    
#### Sample ####
small_num

108

In [108]:
trainDF['texts'] = trainDF['mission_prgrm'].tolist()
trainDF['texts'] = np.array(trainDF['texts'], dtype=object)[:, np.newaxis]

trainDF['label'] = trainDF.NTEE1.astype('category')
category_counts = len(trainDF.label.cat.categories)

trainDF['label'] = np.asarray(pd.get_dummies(trainDF.label), dtype = np.int8)

train_text = trainDF['texts'][:35000]
train_label = trainDF['label'][:35000]

test_text = trainDF['texts'][35000:]
test_label = trainDF['label'][35000:]

category_counts

25

In [109]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
embed_size

512

In [110]:
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)


input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(category_counts, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
lambda_10 (Lambda)           (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_15 (Dense)             (None, 25)                6425      
Total params: 137,753
Trainable params: 137,753
Non-trainable params: 0
_________________________________________________________________


In [111]:
#with tf.Session() as session:
#  K.set_session(session)
#  session.run(tf.global_variables_initializer())
#  session.run(tf.tables_initializer())
history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=10,
            batch_size=1000)

ValueError: Error when checking target: expected dense_15 to have shape (25,) but got array with shape (1,)